In [7]:
import pymc3 as pm
import arviz as az
import numpy as np

# Data Extraction

In [18]:
data_array = np.loadtxt('iop.csv', delimiter=',')
# Extract the IOP data
low_iop_indicators = data_array[:, 0]

# Extract the cornea_thickness data
cornea_thickness = data_array[:, 1]

# Standardize the cornea_thickness data
cornea_thickness_std = (cornea_thickness - cornea_thickness.mean()) / (2 * cornea_thickness.std())

# Model

In [14]:
with pm.Model() as m:
    # Data
    x_data = pm.Data("x_data", cornea_thickness_std) # use standardized data
    y_data = pm.Data("y_data", low_iop_indicators)

    # Priors
    intercept = pm.Normal("intercept", mu=0, sigma=10)
    slope = pm.Normal("slope", mu=0, sigma=5)

    # Logistic function
    p = pm.math.invlogit(intercept + slope * x_data)

    # Likelihood
    pm.Bernoulli("y", p=p, observed=y_data)

    # Sample
    trace = pm.sample(5000, return_inferencedata=False)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [slope, intercept]


/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 6 seconds.


# Show Statistics

In [17]:
az.summary(trace)

Got error No model on context stack. trying to find log_likelihood in translation.
/Users/adamcuculich/georgia-tech/bayesian-stats/env/lib/python3.9/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
intercept,-1.421,0.249,-1.911,-0.975,0.002,0.002,13295.0,12422.0,1.0
slope,-2.213,0.539,-3.215,-1.227,0.005,0.003,13522.0,12247.0,1.0
